In [54]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

In [55]:
food_model = BayesianNetwork([('GroceryShopping', 'MealPrepared'),
                            ('CookingTime','MealPrepared'),
                            ('MealPrepared','Satisfied')])

In [56]:
import numpy as np
values=np.array([[0.5, 0.8, 0.8, 0.9],[0.3, 0.15, 0.1, 0.08],[0.2, 0.05, 0.1, 0.02]])
print(values.shape)

(3, 4)


In [81]:
grocery_cpd = TabularCPD(variable = 'GroceryShopping', variable_card=2, values=[[0.2],[0.8]])

cooking_cpd = TabularCPD(variable = 'CookingTime', variable_card=2, values=[[0.7],[0.3]])

hunger_cpd = TabularCPD(variable = 'Satisfied', variable_card=2, values=[[0.95,0.8,0.5],
                                                                      [0.05,0.2,0.5]], 
                        evidence =['MealPrepared'], evidence_card=[3]) 

mealprep_cpd = TabularCPD(variable = 'MealPrepared', variable_card = 3, values = [[0.5, 0.8, 0.8, 0.9],
                                                                                 [0.3, 0.15, 0.1, 0.08],
                                                                                 [0.2, 0.05, 0.1, 0.02]],
                         evidence= ['GroceryShopping', 'CookingTime'],
                         evidence_card= [2,2])

In [82]:
food_model.add_cpds(grocery_cpd,cooking_cpd,hunger_cpd,mealprep_cpd)

In [83]:
food_model.get_cpds()

[<TabularCPD representing P(GroceryShopping:2) at 0x7f53d759f550>,
 <TabularCPD representing P(CookingTime:2) at 0x7f53d759f280>,
 <TabularCPD representing P(Satisfied:2 | MealPrepared:3) at 0x7f53d759f100>,
 <TabularCPD representing P(MealPrepared:3 | GroceryShopping:2, CookingTime:2) at 0x7f53d759f760>]

## Active Dependencies

In [84]:
food_model.active_trail_nodes('GroceryShopping')

{'GroceryShopping': {'GroceryShopping', 'MealPrepared', 'Satisfied'}}

In [85]:
food_model.active_trail_nodes('MealPrepared')

{'MealPrepared': {'CookingTime',
  'GroceryShopping',
  'MealPrepared',
  'Satisfied'}}

## Local Independencies

In [86]:
food_model.local_independencies('GroceryShopping')

(GroceryShopping ⟂ CookingTime)

In [87]:
food_model.local_independencies('MealPrepared')

In [88]:
food_model.get_independencies()

(Satisfied ⟂ GroceryShopping, CookingTime | MealPrepared)
(Satisfied ⟂ CookingTime | MealPrepared, GroceryShopping)
(Satisfied ⟂ GroceryShopping | MealPrepared, CookingTime)
(GroceryShopping ⟂ CookingTime)
(GroceryShopping ⟂ Satisfied | MealPrepared)
(GroceryShopping ⟂ Satisfied | MealPrepared, CookingTime)
(CookingTime ⟂ GroceryShopping)
(CookingTime ⟂ Satisfied | MealPrepared)
(CookingTime ⟂ Satisfied | MealPrepared, GroceryShopping)

In [89]:
food_infere = VariableElimination(food_model)

In [90]:
prob_hunger = food_infere.query(variables = ['Satisfied'] )
print(prob_hunger)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+--------------+------------------+
| Satisfied    |   phi(Satisfied) |
+==============+==================+
| Satisfied(0) |           0.8898 |
+--------------+------------------+
| Satisfied(1) |           0.1102 |
+--------------+------------------+


In [91]:
prob_satisfied_grocery_bought = food_infere.query(variables = ['Satisfied', 'GroceryShopping'] )
print(prob_satisfied_grocery_bought)
# print(prob_hunger_grocery_bought['Hunger'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------------+--------------------+----------------------------------+
| Satisfied    | GroceryShopping    |   phi(Satisfied,GroceryShopping) |
+==============+====================+==================================+
| Satisfied(0) | GroceryShopping(0) |                           0.1684 |
+--------------+--------------------+----------------------------------+
| Satisfied(0) | GroceryShopping(1) |                           0.7214 |
+--------------+--------------------+----------------------------------+
| Satisfied(1) | GroceryShopping(0) |                           0.0316 |
+--------------+--------------------+----------------------------------+
| Satisfied(1) | GroceryShopping(1) |                           0.0786 |
+--------------+--------------------+----------------------------------+


In [92]:
prob_satisfied_grocery_bought = food_infere.query(variables = ['Satisfied', 'MealPrepared'] )
print(prob_satisfied_grocery_bought)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+-----------------+--------------+-------------------------------+
| MealPrepared    | Satisfied    |   phi(MealPrepared,Satisfied) |
+=================+==============+===============================+
| MealPrepared(0) | Satisfied(0) |                        0.7429 |
+-----------------+--------------+-------------------------------+
| MealPrepared(0) | Satisfied(1) |                        0.0391 |
+-----------------+--------------+-------------------------------+
| MealPrepared(1) | Satisfied(0) |                        0.1010 |
+-----------------+--------------+-------------------------------+
| MealPrepared(1) | Satisfied(1) |                        0.0252 |
+-----------------+--------------+-------------------------------+
| MealPrepared(2) | Satisfied(0) |                        0.0459 |
+-----------------+--------------+-------------------------------+
| MealPrepared(2) | Satisfied(1) |                        0.0459 |
+-----------------+--------------+----------------------------

In [93]:
prob_satisfied_no_grocery = food_infere.query( variables = ['Satisfied'], evidence={'GroceryShopping':0} )
print(prob_satisfied_no_grocery)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------------+------------------+
| Satisfied    |   phi(Satisfied) |
+==============+==================+
| Satisfied(0) |           0.8420 |
+--------------+------------------+
| Satisfied(1) |           0.1580 |
+--------------+------------------+


In [94]:
prob_satisfied_grocery_bought = food_infere.query( variables = ['Satisfied'], evidence={'GroceryShopping':1} )
print(prob_satisfied_grocery_bought)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------------+------------------+
| Satisfied    |   phi(Satisfied) |
+==============+==================+
| Satisfied(0) |           0.9017 |
+--------------+------------------+
| Satisfied(1) |           0.0983 |
+--------------+------------------+


In [95]:
prob_satisfied_grocery_bought_have_cookingTime = food_infere.query( variables = ['Satisfied'], evidence={'GroceryShopping':1, 'CookingTime':1} )
print(prob_satisfied_grocery_bought_have_cookingTime)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+--------------+------------------+
| Satisfied    |   phi(Satisfied) |
+==============+==================+
| Satisfied(0) |           0.9290 |
+--------------+------------------+
| Satisfied(1) |           0.0710 |
+--------------+------------------+


In [100]:
prob_grocery_bought_if_mealPrep_amazing = food_infere.query(variables=['GroceryShopping'],
                                                           evidence={'MealPrepared':2})
print(prob_grocery_bought_if_mealPrep_amazing)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+--------------------+------------------------+
| GroceryShopping    |   phi(GroceryShopping) |
+====================+========================+
| GroceryShopping(0) |                 0.3377 |
+--------------------+------------------------+
| GroceryShopping(1) |                 0.6623 |
+--------------------+------------------------+


In [101]:
prob_grocery_bought_if_no_cookingtime = food_infere.query(variables=['GroceryShopping'], 
                                                          evidence= {'CookingTime':1} )
print(prob_grocery_bought_if_no_cookingtime )

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+--------------------+------------------------+
| GroceryShopping    |   phi(GroceryShopping) |
+====================+========================+
| GroceryShopping(0) |                 0.2000 |
+--------------------+------------------------+
| GroceryShopping(1) |                 0.8000 |
+--------------------+------------------------+


In [102]:
prob_grocery_bought_if_no_cookingtime_good_mealprepared = food_infere.query(
                                                            variables=['GroceryShopping'], 
                                                          evidence= {'CookingTime':1,
                                                                     'MealPrepared':2} )
print(prob_grocery_bought_if_no_cookingtime_good_mealprepared) 

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+--------------------+------------------------+
| GroceryShopping    |   phi(GroceryShopping) |
+====================+========================+
| GroceryShopping(0) |                 0.3846 |
+--------------------+------------------------+
| GroceryShopping(1) |                 0.6154 |
+--------------------+------------------------+


In [103]:
# Most Probable state for a variable

In [104]:
food_infere.map_query(variables=['GroceryShopping'])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

{'GroceryShopping': 1}

In [105]:
food_infere.map_query(variables=['Satisfied'])

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

{'Satisfied': 0}

In [106]:
food_infere.map_query(variables=['MealPrepared'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'MealPrepared': 0}

In [107]:
food_infere.map_query(variables=['CookingTime'])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

{'CookingTime': 0}